# Read and load results

In [2]:
from functions import *
from Classes import *

# Define the use technologies and regions(s)
region = 'ESP'  
setup = {
    f'{region}': {
        'OCGT': True,
        'CCGT': False,
        'battery storage': True,
        'onwind': True,
        'offwind': False,
        'solar': True,
        'electrolysis': True,
        'fuel cell': True,
        'Hydrogen storage': True,
        'Reservoir hydro storage': True,
        'Load shedding': False
    }
}

# Default weather, hydro and demand years
w_year_exp = 2011
h_year_exp = 2007
d_year_exp = 2018

# Dispatch and rolling horizon settings
w_year_dispatch = 2017
h_year_dispatch = 2007
d_year_dispatch = 2019

weather_years = All_data['solar'].index.year.unique()


In [ ]:
folder_rh = f"N_RH_d_{d_year_dispatch}_hMC"
folder_pf = f"N_PF_d_{d_year_dispatch}_hMC"

networks_rh = load_networks(folder_name = folder_rh, weather_years = weather_years)
networks_pf = load_networks(folder_name = folder_pf, weather_years = weather_years)

['N_RHL_w-1979_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1980_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1981_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1982_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1983_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1984_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1985_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1986_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1987_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1988_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1989_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1990_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1991_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1992_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1993_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1994_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1995_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1996_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1997_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1998_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-1999_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-2000_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-2001_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-2002_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-2003_d-2019_h-2007_ESP_hMC',
 'N_RHL_w-2004_d-2019_h-2

# Build df and tables to store results

In [ ]:
if len(names_pf) != len(names_rh):
    print("Warning: The number of loaded PF and RH networks do not match!")

# Results loop (pairs by index order; both lists are alphabetically sorted by filename)
results = []
for i, (name_pf, name_rh) in enumerate(zip(names_pf, names_rh)):
    pf_net = networks_pf[name_pf]
    rh_net = networks_rh[name_rh]

    pf_cost = (pf_net.buses_t['marginal_price']["electricity bus"].values *
               pf_net.loads_t['p_set']['load']).sum() / 1e6
    rh_cost = (rh_net.buses_t['marginal_price']["electricity bus"].values *
               rh_net.loads_t['p_set']['load']).sum() / 1e6
    diff = rh_cost - pf_cost

    # Load shedding cost (present only if the generator exists)
    ls_pf = ((pf_net.generators_t.p["Load shedding"] *
              pf_net.generators.at["Load shedding", "marginal_cost"]).sum() / 1e6
             if "Load shedding" in pf_net.generators.index else 0.0)

    ls_rh = ((rh_net.generators_t.p["Load shedding"] *
              rh_net.generators.at["Load shedding", "marginal_cost"]).sum() / 1e6
             if "Load shedding" in rh_net.generators.index else 0.0)

    results.append({
        'pair_idx': i,
        'PF_key': name_pf,
        'RH_key': name_rh,
        'PF_cost_MEUR': round(pf_cost, 1),
        'RH_cost_MEUR': round(rh_cost, 1),
        'Difference_MEUR': round(diff, 1),
        'PF_Load shedding_cost_MEUR': round(ls_pf, 1),
        'RH_Load shedding_cost_MEUR': round(ls_rh, 1)
    })

results_df = pd.DataFrame(results)


## Functions to calc results

In [11]:
# FOR expansion model: 

def calculate_cost_recovery_generators(network):
    results = []

    for gen in network.generators.index:
        carrier = network.generators.at[gen, "carrier"]
        bus = network.generators.at[gen, "bus"]
        p_nom = network.generators.at[gen, "p_nom_opt"]
        
        marginal_cost = network.generators.at[gen, "marginal_cost"]
        
        capital_cost = network.generators.at[gen, "capital_cost"]
        
        dispatch = network.generators_t.p[gen]
        prices = network.buses_t.marginal_price[bus]

        # Revenue: price * generation
        revenue = (dispatch * prices).sum()

        # Production cost: generation * marginal cost
        production_cost = (dispatch * marginal_cost).sum()

        # Capital cost: installed capacity * capital cost
        capex = p_nom * capital_cost

        # Profit = revenue - (capex + production cost)
        profit = revenue - (capex + production_cost)

        results.append({
            "name": gen,
            "carrier": carrier,
            "revenue [MEUR]": round(revenue / 1e6, 1),
            "production cost [MEUR]": round(production_cost / 1e6, 1),
            "capital cost [MEUR]": round(capex / 1e6, 1),
            "total cost [MEUR]": round((production_cost + capex) / 1e6, 1),
            "profit [MEUR]": round(profit / 1e6, 1)
        })

    df_results = pd.DataFrame(results)
    return df_results
# Example: take the first network from networks_pf
N = networks_pf[names_pf[0]]

CR_N_exp = calculate_cost_recovery_generators(N)
CR_N_exp

,name,carrier,revenue [MEUR],production cost [MEUR],capital cost [MEUR],total cost [MEUR],profit [MEUR]
0,OCGT,gas,3212.5,3212.5,0.0,3212.5,0.0
1,onwind,onwind,5671.4,216.9,0.0,216.9,5454.4
2,solar,solar,1143.8,0.4,0.0,0.4,1143.5
3,Load shedding,load shedding,0.0,0.0,0.0,0.0,0.0


## Look at results

# Plot results

## Result type 1